[Reference](https://python.plainenglish.io/how-i-protect-my-fastapi-apis-from-abuse-6b382e69a371)

In [1]:
pip install slowapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.2 MB/s eta 0:00:00


In [2]:
pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 6.0 MB/s eta 0:00:00


In [3]:
from fastapi import FastAPI, Request
from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded

limiter = Limiter(key_func=get_remote_address)
app = FastAPI()
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

@app.get("/api/data")
@limiter.limit("5/minute")
async def get_data(request: Request):
    return {"message": "This endpoint is rate limited"}

limiter = Limiter(
    key_func=get_remote_address,
    headers_enabled=True
)

In [4]:
from fastapi.responses import JSONResponse
from datetime import datetime

@app.exception_handler(RateLimitExceeded)
async def custom_rate_limit_handler(request: Request, exc: RateLimitExceeded):
    reset_timestamp = getattr(exc, "reset_time", None)
    retry_after = int(reset_timestamp - datetime.now().timestamp()) if reset_timestamp else 60

    return JSONResponse(
        status_code=429,
        content={
            "error": "rate_limit_exceeded",
            "message": "Too many requests. Please try again later.",
            "retry_after_seconds": retry_after,
            "resets_at": datetime.fromtimestamp(reset_timestamp).strftime("%I:%M %p") if reset_timestamp else None,
        },
        headers={
            "Retry-After": str(retry_after),
            "X-RateLimit-Limit": str(exc.detail.limit.amount),
            "X-RateLimit-Remaining": str(exc.detail.remaining),
            "X-RateLimit-Reset": str(int(reset_timestamp)) if reset_timestamp else "",
        },
    )